In [79]:
import numpy as np
import torch
import torch.nn as nn
import pandas as pd
from sklearn.preprocessing import StandardScaler
from torch.utils.data import Dataset, DataLoader

In [80]:
data = pd.read_csv('diabetes.csv')

In [81]:
x = data.iloc[:, 0:-1].values
y = list(data.iloc[:, -1])

In [82]:
y = map(lambda x: 1 if x == 'positive' else 0, y)
y = np.array(list(y), dtype = 'int32')

In [83]:
## feature normalization  (TODO split train validation and test before)
sc = StandardScaler()
x = sc.fit_transform(x)

In [84]:
x = torch.tensor(x)
y = torch.tensor(y).unsqueeze(1)

In [85]:
class Dataset(Dataset):

    def __init__(self,x,y):
        self.x = x
        self.y = y
        
    def __getitem__(self,index):
        # Get one item from the dataset
        return self.x[index], self.y[index]
    
    def __len__(self):
        return len(self.x)

In [86]:
dataset = Dataset(x, y)

In [87]:
train_loader = DataLoader(dataset = dataset, batch_size = 32, shuffle = True)

In [88]:
# build network
class Model(nn.Module):
    def __init__(self, input_features, output_features):
        super(Model, self).__init__()
        self.fc1 = nn.Linear(input_features, 5)
        self.fc2 = nn.Linear(5, 4)
        self.fc3 = nn.Linear(4, 3)
        self.fc4 = nn.Linear(3, output_features)
        self.sigmoid = nn.Sigmoid()
        self.tanh = nn.Tanh()
        
    def forward(self, x):
        out = self.fc1(x)
        out = self.tanh(out)
        out = self.fc2(out)
        out = self.tanh(out)
        out = self.fc3(out)
        out = self.tanh(out)
        out = self.fc4(out)
        out = self.tanh(out)
        out = self.sigmoid(out)
        return out
        

In [89]:
net = Model(7, 1)

# BCE
criterion = nn.BCELoss(size_average = True)
optimizer = torch.optim.SGD(net.parameters(), lr=0.1, momentum=0.9)

/Users/kevinxu/opt/anaconda3/lib/python3.8/site-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='mean' instead.
  warnings.warn(warning.format(ret))


In [94]:
# train
num_epochs = 200
for epoch in range(num_epochs):
    for inputs, labels in train_loader:
        inputs = inputs.float()
        labels = labels.float()
        # FW
        outputs = net(inputs)
        # loss
        loss = criterion(outputs, labels)
        # Clear the gradient buffer
        optimizer.zero_grad()
        # bp
        loss.backward()
        # weight update
        optimizer.step()
        
        # Accuracy
        outputs = (outputs>0.5).float()
        accuracy = (outputs == labels).float().mean()
        print("Epoch {}/{}, Loss: {:.3f}, Accuracy: {:.3f}".format(epoch+1,num_epochs, loss, accuracy))

Epoch 1/200, Loss: 0.511, Accuracy: 0.812
Epoch 1/200, Loss: 0.501, Accuracy: 0.812
Epoch 1/200, Loss: 0.532, Accuracy: 0.781
Epoch 1/200, Loss: 0.498, Accuracy: 0.812
Epoch 1/200, Loss: 0.563, Accuracy: 0.750
Epoch 1/200, Loss: 0.442, Accuracy: 0.875
Epoch 1/200, Loss: 0.378, Accuracy: 0.938
Epoch 1/200, Loss: 0.470, Accuracy: 0.844
Epoch 1/200, Loss: 0.565, Accuracy: 0.750
Epoch 1/200, Loss: 0.501, Accuracy: 0.812
Epoch 1/200, Loss: 0.531, Accuracy: 0.781
Epoch 1/200, Loss: 0.521, Accuracy: 0.781
Epoch 1/200, Loss: 0.484, Accuracy: 0.812
Epoch 1/200, Loss: 0.560, Accuracy: 0.750
Epoch 1/200, Loss: 0.375, Accuracy: 0.938
Epoch 1/200, Loss: 0.409, Accuracy: 0.906
Epoch 1/200, Loss: 0.629, Accuracy: 0.656
Epoch 1/200, Loss: 0.525, Accuracy: 0.781
Epoch 1/200, Loss: 0.407, Accuracy: 0.906
Epoch 1/200, Loss: 0.470, Accuracy: 0.844
Epoch 1/200, Loss: 0.407, Accuracy: 0.906
Epoch 1/200, Loss: 0.598, Accuracy: 0.719
Epoch 1/200, Loss: 0.407, Accuracy: 0.906
Epoch 1/200, Loss: 0.448, Accuracy